In [ ]:
from IPython.display import display_html
from itertools import chain,cycle
import pandas as pd
from keybert import KeyBERT
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [ ]:
doc_other = """
competitor text here
"""
doc_mysite = """
my site text here
"""

In [ ]:

#diversity parameters#
diversity=0.2
use_mmr=True
ngram_range=(4,4)

#for diversity#
#keywords_other = kw_model.extract_keywords(doc_other,keyphrase_ngram_range=ngram_range,use_mmr=use_mmr, diversity=diversity, stop_words=None,top_n = 50)
#df_other = pd.DataFrame(keywords_other, columns=['keyword', 'value_other'])
#keywords_mysite = kw_model.extract_keywords(doc_mysite,keyphrase_ngram_range=ngram_range,use_mmr=use_mmr, diversity=diversity, stop_words=None,top_n = 50)
#df_mysite = pd.DataFrame(keywords_mysite, columns=['keyword', 'value_mysite'])

#plain#
keywords_other = kw_model.extract_keywords(doc_other,keyphrase_ngram_range=ngram_range,stop_words=None,top_n = 50)
df_other = pd.DataFrame(keywords_other, columns=['keyword', 'value_other'])

keywords_mysite = kw_model.extract_keywords(doc_mysite,keyphrase_ngram_range=ngram_range,stop_words=None,top_n = 50)
df_mysite = pd.DataFrame(keywords_mysite, columns=['keyword', 'value_mysite'])

#mysite win#
df_merged= df_other.merge(df_mysite,on='keyword',how='outer').fillna(0)
df_merged['delta']= df_merged['value_other'] - df_merged['value_mysite']

#other win#
df_merged_other_win = df_mysite.merge(df_other,on='keyword',how='outer').fillna(0)
df_merged_other_win['delta']= df_merged_other_win['value_mysite'] - df_merged_other_win['value_other']

pd.set_option('display.max_rows', None)
df_merged.sort_values(by='delta',inplace = True)
df_merged_other_win.sort_values(by='delta',inplace = True)

df_merged=df_merged[df_merged['delta']<0]
df_merged_other_win=df_merged_other_win[df_merged_other_win['delta']<0]
display_side_by_side(df_merged,df_merged_other_win, titles=['mysite win','other win'])

In [ ]:
#paul shapiro's entities#

text_other=entity_create_list(doc_other.lower())
entity_other = pd.DataFrame.from_dict(text_other).sort_values(by='salience',ascending=False)
entity_other = entity_other.groupby(["name"], as_index=False)["salience"].agg({"standev_salience": "std", "sum_salience": "sum", "median_salience": "median", "avg_salience": "mean", "count": "count"}).sort_values(by="count", ascending=False).reset_index()
entity_other['score'] = entity_other.apply(lambda row: create_score(row['avg_salience'], row['count']), axis=1)
entity_other.sort_values(by="score", ascending=False).head()
#entity_other.to_csv('other_entities.csv')
print('other len:')
print(str(len(entity_other)))

text_mysite=entity_create_list(doc_mysite.lower())
entity_mysite = pd.DataFrame.from_dict(text_mysite).sort_values(by='salience',ascending=False)
entity_mysite = entity_mysite.groupby(["name",], as_index=False)["salience"].agg({"standev_salience": "std", "sum_salience": "sum", "median_salience": "median", "avg_salience": "mean", "count": "count"}).sort_values(by="count", ascending=False).reset_index()
entity_mysite['score'] = entity_mysite.apply(lambda row: create_score(row['avg_salience'], row['count']), axis=1)
entity_mysite.sort_values(by="score", ascending=False).head()
#entity_mysite.to_csv('mysite_entities.csv')
print('mysite len:')
print(str(len(entity_mysite)))

####comparison####
entity_mysite=entity_mysite[['name','sum_salience']].rename(columns={'sum_salience': 'sum_salience_mysite'})
entity_other=entity_other[['name','sum_salience']].rename(columns={'sum_salience': 'sum_salience_other'})

#merge
df_merged_other_win = entity_other.merge(entity_mysite,on='name',how='outer').fillna(0)
df_merged_other_win['delta']= df_merged_other_win['sum_salience_mysite'] - df_merged_other_win['sum_salience_other']
df_merged_T_win = entity_mysite.merge(entity_other,on='name',how='outer').fillna(0)
df_merged_T_win['delta']= df_merged_T_win['sum_salience_other'] - df_merged_T_win['sum_salience_mysite']
df_merged_other_win.sort_values(by='delta',inplace = True)
df_merged_T_win.sort_values(by='delta',inplace = True)
df_merged_T_win=df_merged_T_win[df_merged_T_win['delta']<0]
df_merged_other_win=df_merged_other_win[df_merged_other_win['delta']<0]

display_side_by_side(df_merged_T_win,df_merged_other_win, titles=['mysite win','other win'])